In [50]:
# Author: Mattia Silvestri

"""
Main program.
"""

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from utility import PLSInstance, PLSSolver, random_assigner
from models import MyModel
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import csv
import argparse
import time
import pandas as pd

########################################################################################################################

# Set seed in order to reproduce results
tf.random.set_seed(0)

# Tensorflow 2 GPU setup
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

########################################################################################################################

parser = argparse.ArgumentParser()
parser.add_argument("--dim", type=int, default=10, help="Problem dimension")
parser.add_argument("--train", action="store_true",
                    help="Train the model; if not set the default is test mode.", default=False)
parser.add_argument("--test-num", default=None,
                    help="Test identifier.")
parser.add_argument("--num-epochs", default=300, type=int,
                    help="Number of training epochs.")
parser.add_argument("--max-size", default=1000000, type=int,
                    help="Maximum number of training/test instances to be loaded.")
parser.add_argument("--load-mode", default="onehot", choices=["onehot", "string"],
                    help="Dataset loading mode.")
parser.add_argument("--batch-size", default=1024, type=int,
                    help="Mini-batch size.")
parser.add_argument("--leave-columns-domains", action="store_true", default=False,
                    help="True if you don't want to prune columns domains values with forward checking.")
parser.add_argument("--num-sol", type=str, default="10k",
                    help="Number of solutions from which the training set has been generated; thousands are expressed "
                         + "with k (for example 10000=10k).")
parser.add_argument("--model", default="fnn", choices=["fnn", "cnn"],
                    help="Choose the model architecture.")  # TODO: change default
parser.add_argument("--model-type", default="agnostic", choices=["agnostic", "sbrinspiredloss", "negative", "binary"],
                    help="Choose the model type. 'agnostic' is the model-agnostic baseline. 'sbrinspiredloss', "
                         + "'negative' and 'binary' are relatively the mse, negative and binary-cross entropy versions"
                         + " of the SBR inspiredloss.")
parser.add_argument("--validation-size", type=int, default=0,
                    help="Validation set dimension. If zero no validation set is used.")
parser.add_argument("--use-prop", action="store_true", default=False,
                    help="True if you want to assist the estimators with constraints propagation at evaluation time.")
parser.add_argument("--rnd-feas", action="store_true", default=False,
                    help="True if you want to compute feasibility ratio also for random estimator.")
parser.add_argument("--lmbd", default=1.0, type=float, help="Lambda for SBR-inspired term.")
parser.add_argument("--patience", default=10, type=int,
                    help="Specify the number of 10 epochs intervals without improvement in "
                         "feasibility after which training is stopped.")

# args = parser.parse_args()
args = parser.parse_args(["--dim", "7", "--test-num", "pls-7/cnn/all-ts/run-1", "--num-epochs", "10000", "--max-size", "1000000", "--batch-size", "2048", "--num-sol", "10k", "--model", "cnn", "--model-type", "agnostic", "--validation-size", "5000", "--patience", "10"])  # "--train",

# Problem dimension.
DIM = int(args.dim)

COLUMN_TYPES = [int() for _ in range(DIM ** 3)]

# Set training or test mode.
TRAIN = args.train
if TRAIN:
    print("Training mode")
    mode = "train"
else:
    print("Test mode")
    mode = "test"

# Test number identifier
TEST_NUM = args.test_num

# Number of training epochs
EPOCHS = int(args.num_epochs)

# Maximum number of data set examples to load
MAX_SIZE = int(args.max_size)

# Available loading mode are string and one-hot
LOAD_MODE = args.load_mode

# Mini-batch size
BATCH_SIZE = int(args.batch_size)

# True if you want to adopt SRB-inspired loss function
MODEL_TYPE = args.model_type

if mode == "test":
    mode_char = "L"
else:
    mode_char = "B"

if not TRAIN:
    file_name = "pls{}_10k".format(DIM)
else:
    file_name = "pls{}_{}".format(DIM, args.num_sol)

VAL_SIZE = args.validation_size

NUM_SOL = args.num_sol

# Model name for both training and test
model_name = "test-{}/".format(TEST_NUM)

# Where to save plots
SAVE_PATH = "plots/test-{}/".format(TEST_NUM)
try:
    os.makedirs(SAVE_PATH)
except:
    print("Directory already exists")

# Model name for both training and test
model_name = "test-{}/".format(TEST_NUM)

# Where to save plots
SAVE_PATH = "plots/test-{}/".format(TEST_NUM)
try:
    os.makedirs(SAVE_PATH)
except:
    print("Directory already exists")

########################################################################################################################

Test mode
Directory already exists
Directory already exists


In [42]:
# Create a validation set if required
val_indexes = None

if VAL_SIZE > 0:
    print("Loading validation set...")
    start = time.time()
    X_val = pd.read_csv("datasets/pls{}/partial_solutions_{}_train.csv".format(DIM, NUM_SOL),
                        sep=',',
                        header=None,
                        nrows=MAX_SIZE,
                        dtype=np.int8).values

    # Create penalties for the examples
    if MODEL_TYPE != 'agnostic':
        P_val = pd.read_csv("datasets/pls{}/domains_train_{}.csv".format(DIM, NUM_SOL),
                            sep=',',
                            header=None,
                            nrows=MAX_SIZE,
                            dtype=np.int8).values
    else:
        P_val = np.zeros_like(X_val, dtype=np.int8)

    end = time.time()
    print("Elapsed {} seconds".format((end - start)))

    val_indexes = np.random.choice(np.arange(0, X_val.shape[0]), size=VAL_SIZE, replace=False)
    X_val = X_val[val_indexes]
    P_val = P_val[val_indexes]
    validation_set = (X_val, P_val)

# Load training examples
features_filepath = "datasets/pls{}/partial_solutions_{}_{}.csv".format(DIM, NUM_SOL, mode)
print("Loading features from {}...".format(features_filepath))
start = time.time()
X = pd.read_csv(features_filepath, sep=',', header=None, nrows=MAX_SIZE, dtype=np.int8).values
end = time.time()
print("Elapsed {} seconds, {} GB required".format((end - start), X.nbytes / 10 ** 9))
print("Number of rows: {}".format(X.shape[0]))

labels_filepath = "datasets/pls{}/assignments_{}_{}.csv".format(DIM, NUM_SOL, mode)
print("Loading labels from {}...".format(labels_filepath))
start = time.time()
Y = pd.read_csv(labels_filepath, sep=',', header=None, nrows=MAX_SIZE, dtype=np.int32).values
end = time.time()
print("Elapsed {} seconds, {} GB required".format((end - start), Y.nbytes / 10 ** 9))

# Create penalties for the examples
if MODEL_TYPE == 'agnostic' and not args.use_prop:
    P = np.zeros_like(X, dtype=np.int8)
else:
    if not args.leave_columns_domains:
        penalties_filepath = "datasets/pls{}/domains_{}_{}.csv".format(DIM, mode, NUM_SOL)
    else:
        penalties_filepath = "datasets/pls{}/rows_propagation_domains_{}_{}.csv".format(DIM, mode, NUM_SOL)

    print("Loading penalties from {}...".format(penalties_filepath))
    start = time.time()
    P = pd.read_csv(penalties_filepath, sep=',', header=None, nrows=MAX_SIZE, dtype=np.int8).values
end = time.time()
print("Elapsed {} seconds, {} GB required".format((end - start), P.nbytes / 10 ** 9))

# Remove validation samples from the training set
if val_indexes is not None:
    X = np.delete(X, val_indexes, axis=0)
    Y = np.delete(Y, val_indexes, axis=0)
    P = np.delete(P, val_indexes, axis=0)


Loading validation set...
Elapsed 17.54376459121704 seconds
Loading features from datasets/pls7/partial_solutions_10k_train.csv...
Elapsed 19.937610626220703 seconds, 0.122217074 GB required
Number of rows: 356318
Loading labels from datasets/pls7/assignments_10k_train.csv...
Elapsed 0.28601932525634766 seconds, 0.001425272 GB required
Elapsed 0.3430154323577881 seconds, 0.122217074 GB required


In [43]:
# Create TF datasets
dataset = tf.data.Dataset.from_tensor_slices((X, Y, P)).shuffle(10000).batch(BATCH_SIZE)

In [44]:
####### MODEL DEFINITION - models.py

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Reshape
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model


class PLSCNNModel(MyModel):
    def _define_model(self, input_shape):
        model = tf.keras.Sequential()
        model.add(Reshape((7, 7, 7), input_shape=(7**3,)))

        model.add(Conv2D(32, kernel_size=(7, 1), activation='relu', padding='same'))
        model.add(Conv2D(32, kernel_size=(1, 7), activation='relu', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(BatchNormalization())

        model.add(Conv2D(64, (3, 1), activation='relu', padding='same'))
        model.add(Conv2D(64, (1, 3), activation='relu', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(BatchNormalization())

        # model.add(Conv2D(128, (1, 1), activation='relu', padding='same'))
        # model.add(BatchNormalization())

        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(7**3))  # Loss function expects logits

        self.model = model

    def call(self, inputs):
        return self.model(inputs)

In [66]:
# Create the model
if args.model == "cnn":
    model = PLSCNNModel(num_layers=3,
                        num_hidden=[],
                        input_shape=X.shape[1:],
                        output_dim=DIM ** 3,
                        method=MODEL_TYPE, # "agnostic", "sbrinspiredloss", "negative", "binary"
                        lmbd=args.lmbd)
else:
    model = MyModel(num_layers=2,
                    num_hidden=[512, 512],
                    input_shape=X.shape[1:],
                    output_dim=DIM ** 3,
                    method=MODEL_TYPE,
                    lmbd=args.lmbd)


In [46]:
model.model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 7, 7, 7)           0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 32)          1600      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 7, 7, 32)          7200      
_________________________________________________________________
batch_normalization_40 (Batc (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 7, 7, 64)          6208      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 7, 7, 64)          12352     
_________________________________________________________________
batch_normalization_41 (Batc (None, 7, 7, 64)         

In [47]:
model.predict(X[:1].astype(float))

array([[-4.73650359e-03, -2.01584622e-02,  2.09986954e-03,
         7.17998110e-03,  1.18609499e-02, -5.47342375e-03,
        -6.72534760e-03, -7.34255556e-03, -1.48158185e-02,
        -1.63298603e-02,  9.69978422e-03,  1.25913043e-02,
        -1.03253992e-02, -1.75762437e-02, -1.87293813e-02,
         1.19368145e-02, -1.63152465e-03, -9.23205726e-03,
         4.74093156e-03,  6.68801554e-03, -1.32867601e-02,
        -9.90432035e-03,  1.04816537e-02, -1.34730088e-02,
        -1.34357205e-03,  6.54482655e-03,  4.57880180e-03,
         6.49451651e-03, -4.47922666e-03,  3.14505259e-03,
         7.75613589e-03, -1.85543555e-03,  1.82329789e-02,
         1.48885222e-02,  9.70343780e-03,  1.10105600e-03,
        -7.22034089e-03, -6.86610863e-03, -1.36830984e-02,
         1.24950772e-02,  1.29224639e-02, -3.75703466e-03,
         1.62678063e-02,  4.24929056e-03, -9.35606472e-03,
        -8.69772490e-03, -1.72871840e-03,  1.28156980e-02,
         9.83717944e-03,  1.12139643e-03, -3.83308018e-0

In [67]:
# Train model
if TRAIN:
    history = model.train(EPOCHS,
                          dataset,
                          "models/{}".format(model_name) + str(np.random.randint(1, 10000)),
                          DIM,
                          validation_set,
                          args.use_prop,
                          args.patience)

    for name in history.keys():
        values = history[name]

        plt.plot(np.arange(0, len(values)), values,
                 label=name)
        plt.ylim(bottom=0)
        plt.legend()
        plt.savefig("{}/{}.png".format(SAVE_PATH, name))
        plt.close()

        with open("{}/{}.csv".format(SAVE_PATH, name), "w") as file:
            wr = csv.writer(file)
            wr.writerow(values)
            file.close()
    exit(0)

else:
    model.model = tf.saved_model.load("models/{}".format(model_name))


In [68]:
X_tensor = X.astype(np.float32)
X_tensor = tf.convert_to_tensor(X_tensor, dtype=tf.float32)

infer = model.model.signatures["serving_default"]
# Make inference
pred_tensor = infer(X_tensor)


In [69]:
print(pred_tensor)

{'dense_3': <tf.Tensor: shape=(351318, 343), dtype=float32, numpy=
array([[-2.5986722 , -1.6797662 , -1.0459328 , ..., -1.218635  ,
        -0.10315448, -1.6703275 ],
       [-3.6340992 , -2.3993962 , -1.6982248 , ..., -1.1122572 ,
         0.04810607, -1.272087  ],
       [-3.2417114 , -2.3754346 , -1.9868344 , ..., -0.79298204,
         0.15775168, -0.93916833],
       ...,
       [-0.0761533 ,  0.07599507,  0.01654844, ..., -0.16565886,
        -0.08524382, -0.21267867],
       [-0.10730623,  0.02281542, -0.04375248, ..., -0.0653744 ,
        -0.05370036, -0.13454117],
       [-0.02866615,  0.03528141, -0.02712772, ..., -0.09784694,
        -0.07933479, -0.17733492]], dtype=float32)>}


In [70]:

# Inference output is a dictionary; last layer is the output one
pred_tensor = pred_tensor["dense_{}".format(model.num_layers)]

In [71]:
################################################################################

# Test the model

# Make predictions
tensor_X = X.astype(np.float32)
predict_val = model.predict_from_saved_model(tensor_X).numpy()

# Prune values according to constraints propagator if required
if args.use_prop:
    predict_val *= (1 - P)

# Count of correct predictions grouped by number of assigned variables
pred_by_num_assigned = np.zeros(shape=(DIM ** 2))
# Count of feasible solutions grouped by number of assigned variables
feas_by_num_assigned = np.zeros(shape=(DIM ** 2))
# Count of total examples grouped by number of assigned variables
tot_by_num_assigned = np.zeros(shape=(DIM ** 2))
# Count of random correct predictions grouped by number of assigned variables
rand_pred_by_num_assigned = np.zeros(shape=(DIM ** 2))
# Count of random feasible solutions grouped by number of assigned variables
rand_feas_by_num_assigned = np.zeros(shape=(DIM ** 2))


<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x0000020F00BC0708>


In [72]:
# Compute overall accuracy on training set
acc = 0
count = 0
acc_rand = 0

# Compute accuracy grouped by number of assigned variables
preds = []
for x, pred, y, d in zip(X, predict_val, Y, P):

    if count % 1000 == 0:
        print("Examined {} instances".format(count))

    num_assigned_vars = np.sum(x.astype(np.int8))
    pred_label = np.argmax(pred.reshape(-1))
    correct_label = np.argmax(y.reshape(-1))

    if pred_label == correct_label:
        acc += 1
        pred_by_num_assigned[num_assigned_vars] += 1

    # Create a problem instance with current examples for net prediction
    square = np.reshape(x, (DIM, DIM, DIM))
    pls = PLSInstance(n=DIM)
    pls.square = square.copy()
    # assert pls.__check_constraints__(), "Constraints should be verified before assignment"

    # Make the prediction assignment
    assignment = np.argmax(pred)
    assignment = np.unravel_index(assignment, shape=(DIM, DIM, DIM))

    # Local consistency
    local_feas = pls.assign(assignment[0], assignment[1], assignment[2])

    '''vals_square = np.argmax(square, axis=2) + np.sum(square, axis=2)
    solver = utility.PLSSolver(DIM, square=np.reshape(vals_square, -1))
    res = solver.solve()
    assert res, "Constraint solver is wrong because the input comes from a real solution"'''

    # Global consistency
    if local_feas:
        vals_square = np.argmax(pls.square.copy(), axis=2) + np.sum(pls.square.copy(), axis=2)
        solver = PLSSolver(DIM, square=np.reshape(vals_square, -1))
        feas = solver.solve()
    else:
        feas = local_feas

    if feas:
        feas_by_num_assigned[num_assigned_vars] += 1

    # Check random assignment performance if required
    if args.rnd_feas:
        if not args.use_prop:
            d = None
        rand_assignment = random_assigner(DIM ** 3, d)
        if rand_assignment == correct_label:
            acc_rand += 1
            rand_pred_by_num_assigned[num_assigned_vars] += 1

        # Create a problem instance with current training example for random prediction
        square = np.reshape(x, (DIM, DIM, DIM))
        pls = PLSInstance(n=DIM)
        pls.square = square.copy()
        # assert pls.__check_constraints__(), "Constraints should be verified before assignment"

        # Make the random assignment
        rand_assignment = np.unravel_index(rand_assignment, shape=(DIM, DIM, DIM))

        local_feas = pls.assign(rand_assignment[0], rand_assignment[1], rand_assignment[2])

        # Check global consistency
        if local_feas:
            vals_square = np.argmax(pls.square.copy(), axis=2) + np.sum(pls.square.copy(), axis=2)
            solver = PLSSolver(DIM, square=np.reshape(vals_square, -1))
            feas = solver.solve()
        else:
            feas = local_feas

        if feas:
            rand_feas_by_num_assigned[num_assigned_vars] += 1

    # Increase count of solutions with this number of assignments
    tot_by_num_assigned[num_assigned_vars] += 1
    count += 1

    # Save results checkpoint
    if count % 1000 == 0:

        feasibility = list((feas_by_num_assigned / (tot_by_num_assigned + 1e-8))[1:])

        if not args.use_prop:
            filename = "{}/feasibility_{}.csv".format(SAVE_PATH, mode)
        else:
            if args.leave_columns_domains:
                filename = "{}/feasibility_{}_with_row_prop.csv".format(SAVE_PATH, mode)
            else:
                filename = "{}/feasibility_{}_with_full_prop.csv".format(SAVE_PATH, mode)

        with open(filename, "w") as epoch_file:
            wr = csv.writer(epoch_file)
            wr.writerow(feasibility)

Examined 0 instances
Examined 1000 instances
Examined 2000 instances
Examined 3000 instances
Examined 4000 instances
Examined 5000 instances
Examined 6000 instances
Examined 7000 instances
Examined 8000 instances
Examined 9000 instances
Examined 10000 instances
Examined 11000 instances
Examined 12000 instances
Examined 13000 instances
Examined 14000 instances
Examined 15000 instances
Examined 16000 instances
Examined 17000 instances
Examined 18000 instances
Examined 19000 instances
Examined 20000 instances
Examined 21000 instances
Examined 22000 instances
Examined 23000 instances
Examined 24000 instances
Examined 25000 instances
Examined 26000 instances
Examined 27000 instances
Examined 28000 instances
Examined 29000 instances
Examined 30000 instances
Examined 31000 instances
Examined 32000 instances
Examined 33000 instances
Examined 34000 instances
Examined 35000 instances
Examined 36000 instances
Examined 37000 instances
Examined 38000 instances
Examined 39000 instances
Examined 4000

In [73]:
# Check accuracy is correctly computed
assert np.sum(pred_by_num_assigned) == acc and np.sum(tot_by_num_assigned) == count, \
    "acc: {} | acc_vectorized: {} | count: {} | count_vectorized: {}".format(acc, np.sum(pred_by_num_assigned),
                                                                             count, np.sum(tot_by_num_assigned))

In [74]:
# Make plots

accuracy = list((pred_by_num_assigned / (tot_by_num_assigned + 1e-8))[1:])
feasibility = list((feas_by_num_assigned / (tot_by_num_assigned + 1e-8))[1:])
if args.rnd_feas:
    random_feasibility = list((rand_feas_by_num_assigned / (tot_by_num_assigned + 1e-8))[1:])

In [77]:
feasibility

[0.9912280701464553,
 0.9804709936803842,
 0.9721366376410286,
 0.9631459473882186,
 0.9541014318199463,
 0.9395964185474548,
 0.9253372512342389,
 0.9132004814753066,
 0.8969348146153167,
 0.8884275381718613,
 0.8655282817491121,
 0.8473691257185066,
 0.8312817768252191,
 0.8003207698465646,
 0.7753516409902541,
 0.7368703108243075,
 0.6994258245417286,
 0.6499866202827803,
 0.6063076306286165,
 0.5496050341403808,
 0.5040737277941711,
 0.45188060500541843,
 0.41613162118724434,
 0.3863758029973416,
 0.37553418803368654,
 0.3684632874142457,
 0.36673346693337777,
 0.3660881174894979,
 0.3763354700849675,
 0.3803099118349181,
 0.3920545746383199,
 0.4037791476810456,
 0.41264536826572296,
 0.42616372391596286,
 0.44569138276493564,
 0.45412047549024426,
 0.46924829157112524,
 0.4875568637938352,
 0.5188401924097824,
 0.5476762820505506,
 0.5759882478624787,
 0.6101242153057164,
 0.6486919380664059,
 0.6881533101036074,
 0.7433391350907171,
 0.7875668449187332,
 0.8427731765951874,
 0.8

In [75]:
# Save random assigner results
if args.rnd_feas:
    RANDOM_SAVE_PATH = "plots/test-pls-{}-tf-keras/random/".format(DIM)

    if args.use_prop:
        if not args.leave_columns_domains:
            RANDOM_SAVE_PATH += "rows-and-columns-prop"
        else:
            RANDOM_SAVE_PATH += "rows-prop"
    else:
        RANDOM_SAVE_PATH += "no-prop"

    try:
        os.makedirs(RANDOM_SAVE_PATH)
    except:
        print("Directory {} already exists".format(RANDOM_SAVE_PATH))

    with open("{}/random_feasibility.csv".format(RANDOM_SAVE_PATH, mode), "w") as epoch_file:
        wr = csv.writer(epoch_file)
        wr.writerow(random_feasibility)
